# Data Extraction To DataBase Structuring
Cette section du projet consiste à extraire les données relatives au Bénin depuis la base GDELT, à les transformer, puis à les intégrer dans une base de données relationnelle.
Cette structuration permet une consultation rapide et ciblée des événements, la réalisation d’analyses croisées complexes, ainsi que l’intégration avec d'autres outils analytiques, y compris des solutions d’intelligence artificielle.

## Data Extraction
A cette étape, l’objectif est d’extraire des données spécifiques au Bénin à partir de la base de données GDELT (Global Database of Events, Language, and Tone).
La base GDELT recense quotidiennement des événements médiatiques mondiaux décrivant des interactions entre acteurs (pays, groupes, individus), les lieux impliqués, les tonalités médiatiques et le type d’événement.

Cette extraction constitue la base de travail pour les étapes suivantes de structuration et d’analyse.


In [1]:
%pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage pandas pyarrow db-dtypes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from functions.initialize_client import initialize_bigquery_client
from functions.setup_authentication import setup_authentication

# Définir le chemin vers le fichier de clé d'authentification BigQuery
bigquery_key_path = "/home/pionner02/Pionner02 UlChris-Project/pionner02-ulchris-457521-key.json"
auth_env = "GOOGLE_APPLICATION_CREDENTIALS"

# Définir la variable d'environnement pour authentifier l'accès à Google Cloud avec la clé
setup_authentication(bigquery_key_path, auth_env)

# Initialisation le client BigQuery en utilisant la clé d'authentification configurée précédemment
project_id = "pionner02-ulchris-457521"
client = initialize_bigquery_client(project_id)

2025-04-21 22:15:31,936 - INFO - Clé d'authentification chargée dans GOOGLE_APPLICATION_CREDENTIALS : /home/pionner02/Pionner02 UlChris-Project/pionner02-ulchris-457521-key.json
2025-04-21 22:15:31,937 - INFO - Aucun identifiant explicite, utilisation des Application Default Credentials
2025-04-21 22:15:31,976 - INFO - Client BigQuery initialisé pour le projet pionner02-ulchris-457521


In [7]:
import os

# Définir le dossier de destination des fichiers de données CSV
output_csv_dir = "/home/pionner02/Pionner02 UlChris-Project/data/extract_data"

os.makedirs(output_csv_dir, exist_ok=True)  # Créer le dossier si inexistant
print(f" Les fichiers CSV seront sauvegardés dans : {output_csv_dir}")

 Les fichiers CSV seront sauvegardés dans : /home/pionner02/Pionner02 UlChris-Project/data/extract_data


In [9]:
from functions.extract_table import extract_table

# Requêtes SQL BigQuery

# Requête sur les événements
events_query= """
SELECT  GLOBALEVENTID, 
        SQLDATE, 
        GoldsteinScale,
        NumMentions, 
        NumSources, 
        NumArticles, 
        AvgTone,
        ActionGeo_FullName,
        ActionGeo_Lat,
        ActionGeo_Long,
        SOURCEURL
FROM `gdelt-bq.gdeltv2.events_partitioned`
WHERE ActionGeo_CountryCode = 'BN' 
    OR Actor1Geo_CountryCode = 'BN' 
    OR Actor2Geo_CountryCode = 'BN' 
    OR Actor1CountryCode = 'BEN' 
    OR Actor2CountryCode = 'BEN'
    """
# Requête sur les mentions liées aux événements    
mentions_query="""
SELECT  m.GLOBALEVENTID, 
        m.MentionTimeDate, 
        m.MentionType,
        m.MentionSourceName,
        m.MentionIdentifier,
        m.ActionCharOffset,
        m.Confidence,
        m.MentionDocTone
FROM `gdelt-bq.gdeltv2.eventmentions_partitioned` m
INNER JOIN `gdelt-bq.gdeltv2.events_partitioned` e
        ON m.GLOBALEVENTID = e.GLOBALEVENTID
WHERE ActionGeo_CountryCode = 'BN' 
    OR e.Actor1Geo_CountryCode = 'BN' 
    OR e.Actor2Geo_CountryCode = 'BN' 
    OR e.Actor1CountryCode = 'BEN' 
    OR e.Actor2CountryCode = 'BEN'
    """

In [ ]:
# Exécution de toutes les extractions définies

extracted_files = {}    # Dictionnaire pour stocker les fichiers générés

# Liste des extractions à effectuer
try:
    for query, table_name in [
        (events_query, "benin_events"),
        (mentions_query, "benin_mentions"),
    ]:
        try:
            print(f" Extraction en cours pour {table_name}")
            output_file = extract_table(
                query=query,
                table_name=table_name,
                output_dir=output_csv_dir,
                client=client
            )
            if output_file:
                print(f" Extraction réussie pour {table_name}")
                extracted_files[table_name] = output_file
            else:
                print(f" Pas de fichier généré pour {table_name}")
        except Exception as e:
            print(f" Échec de l'extraction pour {table_name} : {e}")

    print(f" Les extractions sont terminées. Données enregistrées dans {output_file}")
    if extracted_files:
        print(" Fichiers générés après extraction :")
        for name, path in extracted_files.items():
            print(f"  - Fichier {name} sauvegardé dans {path}")
    else:
        print(" Aucune donnée n’a été extraite avec succès.")

finally:
    client.close()
    print(" Client BigQuery fermé proprement.")

2025-04-21 22:18:27,435 - INFO - Début de l'extraction pour la table 'benin_events'


 Extraction en cours pour 
SELECT  GLOBALEVENTID, 
        SQLDATE, 
        GoldsteinScale,
        NumMentions, 
        NumSources, 
        NumArticles, 
        AvgTone,
        ActionGeo_FullName,
        ActionGeo_Lat,
        ActionGeo_Long,
        SOURCEURL
FROM `gdelt-bq.gdeltv2.events_partitioned`
WHERE ActionGeo_CountryCode = 'BN' 
    OR Actor1Geo_CountryCode = 'BN' 
    OR Actor2Geo_CountryCode = 'BN' 
    OR Actor1CountryCode = 'BEN' 
    OR Actor2CountryCode = 'BEN'
    


2025-04-21 22:18:36,881 - INFO - Octets facturés : 175557836800
2025-04-21 22:18:38,736 - INFO - Les données ont été exportées dans le fichier : benin_events_20250421_221827.csv
2025-04-21 22:18:38,737 - INFO - Nombre de lignes dans le fichier : 468642
2025-04-21 22:18:38,745 - INFO - Début de l'extraction pour la table 'benin_mentions'


 Extraction réussie pour 
SELECT  GLOBALEVENTID, 
        SQLDATE, 
        GoldsteinScale,
        NumMentions, 
        NumSources, 
        NumArticles, 
        AvgTone,
        ActionGeo_FullName,
        ActionGeo_Lat,
        ActionGeo_Long,
        SOURCEURL
FROM `gdelt-bq.gdeltv2.events_partitioned`
WHERE ActionGeo_CountryCode = 'BN' 
    OR Actor1Geo_CountryCode = 'BN' 
    OR Actor2Geo_CountryCode = 'BN' 
    OR Actor1CountryCode = 'BEN' 
    OR Actor2CountryCode = 'BEN'
    
 Extraction en cours pour 
SELECT  m.GLOBALEVENTID, 
        m.MentionTimeDate, 
        m.MentionType,
        m.MentionSourceName,
        m.MentionIdentifier,
        m.ActionCharOffset,
        m.Confidence,
        m.MentionDocTone
FROM `gdelt-bq.gdeltv2.eventmentions_partitioned` m
INNER JOIN `gdelt-bq.gdeltv2.events_partitioned` e
        ON m.GLOBALEVENTID = e.GLOBALEVENTID
WHERE ActionGeo_CountryCode = 'BN' 
    OR e.Actor1Geo_CountryCode = 'BN' 
    OR e.Actor2Geo_CountryCode = 'BN' 
    OR e.

2025-04-21 22:18:54,276 - INFO - Octets facturés : 425344368640
2025-04-21 22:18:56,687 - INFO - Les données ont été exportées dans le fichier : benin_mentions_20250421_221838.csv
2025-04-21 22:18:56,688 - INFO - Nombre de lignes dans le fichier : 796704


 Extraction réussie pour 
SELECT  m.GLOBALEVENTID, 
        m.MentionTimeDate, 
        m.MentionType,
        m.MentionSourceName,
        m.MentionIdentifier,
        m.ActionCharOffset,
        m.Confidence,
        m.MentionDocTone
FROM `gdelt-bq.gdeltv2.eventmentions_partitioned` m
INNER JOIN `gdelt-bq.gdeltv2.events_partitioned` e
        ON m.GLOBALEVENTID = e.GLOBALEVENTID
WHERE ActionGeo_CountryCode = 'BN' 
    OR e.Actor1Geo_CountryCode = 'BN' 
    OR e.Actor2Geo_CountryCode = 'BN' 
    OR e.Actor1CountryCode = 'BEN' 
    OR e.Actor2CountryCode = 'BEN'
    
 Les extractions sont terminées. Données enregistrées dans /home/pionner02/Pionner02 UlChris-Project/data/extract_data/benin_mentions_20250421_221838.csv
 Fichiers générés après extraction :
  - Fichier benin_events sauvegardé dans /home/pionner02/Pionner02 UlChris-Project/data/extract_data/benin_events_20250421_221827.csv
  - Fichier benin_mentions sauvegardé dans /home/pionner02/Pionner02 UlChris-Project/data/extract_dat

## DataBase Structuring
À la suite de l’extraction des données relatives au Bénin depuis la base GDELT, cette étape consiste à préparer et structurer les données dans une base de données relationnelle.

Avant l’intégration dans la base de données, les données extraites subissent une phase de préparation indispensable afin d’assurer leur qualité, leur cohérence et leur compatibilité avec le modèle relationnel. Cette phase garantit que les données sont fiables, uniformes et prêtes à être intégrées dans une base relationnelle sans erreur technique ou logique.

Quant à la structuration, elle a pour objectif de rendre les données plus accessibles, faciles à interroger de manière efficace, et prêtes pour des analyses avancées.
Elle repose sur une organisation logique et normalisée des informations extraites, facilitant leur exploitation future dans des environnements analytiques.

In [2]:
# Dossier des fichiers CSV après extraction
output_csv_dir = "/home/pionner02/Pionner02 UlChris-Project/data/extract_data"

In [ ]:
from functions.get_latest_file import get_latest_file_by_keyword

# Identification des fichiers CSV les plus récents extraits de GDELT
latest_files_dict = {}

try:
    for prefix in [
                  'benin_events', 
                  'benin_mentions'
                 ]:
        latest_file = get_latest_file_by_keyword(prefix, output_csv_dir, ".csv")
        if latest_file:
            latest_files_dict[prefix] = latest_file
        
except Exception as e:
    print(f" Une erreur est survenue lors de la recherche : {e}")

 Fichier le plus récent trouvé dans /home/pionner02/Pionner02 UlChris-Project/data/extract_data pour 'benin_events' : benin_events_20250421_221827.csv
 Fichier le plus récent trouvé dans /home/pionner02/Pionner02 UlChris-Project/data/extract_data pour 'benin_mentions' : benin_mentions_20250421_221838.csv


In [48]:
# Définir le dossier de destination des fichiers après nettoyage des données
cleaned_csv_dir = "/home/pionner02/Pionner02 UlChris-Project/data/cleaned_data"

os.makedirs(cleaned_csv_dir, exist_ok=True)  # Créer le dossier si inexistant
print(f" Les fichiers CSV après traitement des données seront sauvegardés dans : {cleaned_csv_dir}")

# Dossier contenant les fichiers extraits de GDELT
input_dir = "/home/pionner02/Pionner02 UlChris-Project/data/extract_data"

 Les fichiers CSV après traitement des données seront sauvegardés dans : /home/pionner02/Pionner02 UlChris-Project/data/cleaned_data


In [49]:
import pandas as pd

dfs = {}  # Dictionnaire pour stocker les DataFrames à partir des fichiers exportés

for prefix, file_name in latest_files_dict.items():
    print(f"Ficher trouvé pour {prefix=} : {file_name=}")

    if file_name is None:
        print(f" Aucun fichier trouvé pour le préfixe '{prefix}'.")
        continue
    
    if not file_name.lower().endswith('.csv'):
        print(f" Le fichier '{file_name}' n'est pas un CSV.")
        continue
    
    input_path = os.path.join(input_dir, file_name)   # Dossier d'origine des fichiers à traiter

    print(f"Chargement du fichier : {file_name}")
    
    try:
        df = pd.read_csv(input_path)
        dfs[prefix] = {
        "df": df,
        "file_name": file_name
        }
        
        print(f"Fichier {file_name} chargé : ({len(df)} lignes)")
    except FileNotFoundError:
        print(f"Fichier introuvable : {input_path}")
        continue
        
    except Exception as e:
        print(f"Erreur lors du chargement de {file_name} : {e}")
        continue

Ficher trouvé pour prefix='benin_events' : file_name='benin_events_20250421_221827.csv'
Chargement du fichier : benin_events_20250421_221827.csv
Fichier benin_events_20250421_221827.csv chargé : (468642 lignes)
Ficher trouvé pour prefix='benin_mentions' : file_name='benin_mentions_20250421_221838.csv'
Chargement du fichier : benin_mentions_20250421_221838.csv
Fichier benin_mentions_20250421_221838.csv chargé : (796704 lignes)


In [ ]:
from IPython.display import display

for prefix, data in dfs.items():
    df = data["df"]
    file_name = data["file_name"]

   # Aperçu des données contenues dans les fichiers chargés
    try:
        print(f"Aperçu des premières de lignes {file_name} :")
        display(df.head())
        
    except Exception as e:
        print(f"Erreur lors l'affichage de {file_name} :", e)

Aperçu des premières de lignes benin_events_20250421_221827.csv :


,GLOBALEVENTID,SQLDATE,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
0,1238864555,20250419,-10.0,15,2,15,-9.746639,"Koudou, Benin (general), Benin",10.8945,1.16328,https://guardian.ng/news/suspected-jihadists-k...
1,1239269408,20250421,-10.0,16,2,16,-3.942652,Benin,9.5000,2.25000,https://www.aljazeera.net/news/2025/4/21/%D8%A...
2,1117208340,20230726,-2.0,30,3,30,-4.848485,Benin,9.5000,2.25000,https://www.varmatin.com/faits-divers/qui-etai...
3,1117210338,20230726,3.4,12,1,12,4.403409,Benin,9.5000,2.25000,https://www.prnewswire.com/news-releases/nexge...
4,1124580946,20230903,3.4,12,2,12,2.535821,Benin,9.5000,2.25000,https://www.naijanews.com/2023/09/03/edo-obase...


Aperçu des premières de lignes benin_mentions_20250421_221838.csv :


,GLOBALEVENTID,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,ActionCharOffset,Confidence,MentionDocTone
0,1080064261,20230113200000,1,bbc.com,https://www.bbc.com/news/av/world-africa-64269446,71,90,6.382979
1,1176169626,20240516184500,1,tribuneonlineng.com,https://tribuneonlineng.com/edo-2024-inec-to-b...,417,90,1.176471
2,1001321613,20210827211500,1,ahram.org.eg,https://english.ahram.org.eg/NewsContent/1/64/...,367,90,1.923077
3,1080210460,20230115143000,1,english.news.cn,https://english.news.cn/20230115/8600befffbc44...,391,90,7.572816
4,1080210533,20230115143000,1,english.news.cn,https://english.news.cn/20230115/8600befffbc44...,391,90,7.572816


In [51]:
missing_data_by_table = {}  # Stocker les lignes ayant des valeurs manquantes

for prefix, data in dfs.items():
    df = data["df"]
    file_name = data["file_name"]

    try:
        print(f"    - Fichier {file_name} : ")
        
        # Affichage des valeurs manquantes
        missing_values = df.isnull().sum()
        print(f"Valeurs manquantes par colonne : \n{missing_values}")

        missing_rows = df[df.isnull().any(axis=1)]
        print(f"Nombre de lignes avec des valeurs manquantes :", len(missing_rows))
        
        # Affichage des lignes avec valeurs manquantes
        if not missing_rows.empty:
            missing_data_by_table[prefix] = missing_rows
            print("Aperçu des lignes avec valeurs manquantes :")
            display(missing_rows.head())
        
    except Exception as e:
        print(f"Erreur lors de l'affichage des valeurs manquantes de {file_name} :", e)

    - Fichier benin_events_20250421_221827.csv : 
Valeurs manquantes par colonne : 
GLOBALEVENTID            0
SQLDATE                  0
GoldsteinScale          22
NumMentions              0
NumSources               0
NumArticles              0
AvgTone                  0
ActionGeo_FullName    3638
ActionGeo_Lat         3289
ActionGeo_Long        3284
SOURCEURL                0
dtype: int64
Nombre de lignes avec des valeurs manquantes : 3660
Aperçu des lignes avec valeurs manquantes :


,GLOBALEVENTID,SQLDATE,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
273,745951159,20180409,0.4,15,1,15,0.000000,NaN,NaN,NaN,https://www.cta.int/en/copyrights
1016,648170050,20170419,3.0,30,1,30,-2.572721,NaN,NaN,NaN,http://www.24haubenin.info/spip.php?page=sum&i...
1017,648170051,20170419,7.0,30,1,30,-2.572721,NaN,NaN,NaN,http://www.24haubenin.info/spip.php?page=sum&i...
2223,753468099,20180506,3.4,30,1,30,-2.007306,NaN,NaN,NaN,https://www.24haubenin.info/spip.php?page=sum&...
3455,681438471,20170815,1.9,30,1,30,0.389107,NaN,NaN,NaN,https://www.24haubenin.info/spip.php?page=sum&...


    - Fichier benin_mentions_20250421_221838.csv : 
Valeurs manquantes par colonne : 
GLOBALEVENTID         0
MentionTimeDate       0
MentionType           0
MentionSourceName    90
MentionIdentifier     0
ActionCharOffset      0
Confidence            0
MentionDocTone        0
dtype: int64
Nombre de lignes avec des valeurs manquantes : 90
Aperçu des lignes avec valeurs manquantes :


,GLOBALEVENTID,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,ActionCharOffset,Confidence,MentionDocTone
34254,417988872,20150318043000,1,NaN,http://162.218.139.66/daily/politics/49674-as-...,8502,10,-1.408451
34814,411293032,20150222080000,1,NaN,http://162.218.139.66/sunday/index.php/point-b...,4041,10,0.664767
104447,418341268,20150319063000,1,NaN,http://162.218.139.66/daily/news/49870-oshiomh...,898,10,-8.626198
104448,418341275,20150319063000,1,NaN,http://162.218.139.66/daily/news/49870-oshiomh...,934,10,-8.626198
110627,416618278,20150313143000,1,NaN,http://162.218.139.66/daily/top-stories/49389-...,5282,20,-2.270148


In [52]:
duplicate_data_by_table = {}    # Stocker les lignes en double
    
for prefix, data in dfs.items():
    df = data["df"]
    file_name = data["file_name"]
 
    try:
        print(f"    - Fichier {file_name} : ")
        
        # Affichage des doublons    
        duplicate_rows = df[df.duplicated(keep=False)]
        print(f"Nombre de lignes en doublons : {len(duplicate_rows)}")

        # Affichage des lignes en doubles
        if not duplicate_rows.empty:
            duplicate_data_by_table[prefix] = duplicate_rows
            print("Aperçu des lignes en doubles :")
            display(duplicate_rows.head())
            
    except Exception as e:
        print(f"Erreur de l'affichage des lignes en doubles de {file_name} :", e)

    - Fichier benin_events_20250421_221827.csv : 
Nombre de lignes en doublons : 2
Aperçu des lignes en doubles :


,GLOBALEVENTID,SQLDATE,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
462778,542707439,20160422,7.0,10,1,10,-2.265372,Benin,9.5,2.25,http://www.brecorder.com/cotton-a-textiles/185...
462782,542707439,20160422,7.0,10,1,10,-2.265372,Benin,9.5,2.25,http://www.brecorder.com/cotton-a-textiles/185...


    - Fichier benin_mentions_20250421_221838.csv : 


Nombre de lignes en doublons : 6996
Aperçu des lignes en doubles :


,GLOBALEVENTID,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,ActionCharOffset,Confidence,MentionDocTone
46,517495445,20160303203000,1,nairaland.com,http://www.nairaland.com/2966852/how-became-he...,17286,90,1.2507
178,1152035465,20240117041500,1,newsverge.com,https://newsverge.com/2024/01/16/obaseki-to-ho...,398,90,0.0000
179,1152035465,20240117041500,1,newsverge.com,https://newsverge.com/2024/01/16/obaseki-to-ho...,398,90,0.0000
180,1152056639,20240117070000,1,thenationonlineng.net,https://thenationonlineng.net/southsouth-gover...,394,90,0.0000
181,1152056639,20240117070000,1,thenationonlineng.net,https://thenationonlineng.net/southsouth-gover...,394,90,0.0000


In [53]:
for prefix, data in dfs.items():
    df = data["df"]
    file_name = data["file_name"]

    try:
        print(f"\n Nettoyage du fichier : {file_name}")

        # Suppression des valeurs manquantes
        original_len = len(df)
        df = df.dropna()
        print(f" Lignes supprimées (valeurs manquantes) : {original_len - len(df)} (reste {len(df)})")

        # Suppression des doublons
        original_len = len(df)
        df = df.drop_duplicates()
        print(f" Lignes supprimées (doublons) : {original_len - len(df)} (reste {len(df)})")

        # Définir nom du fichier nettoyé
        base_name, ext = os.path.splitext(file_name)  # séparer nom et extension
        cleaned_file_name = f"{base_name}_cleaned{ext}"

        # Destination des fichiers après nettoyage
        cleaned_output_path = os.path.join(cleaned_csv_dir, cleaned_file_name)
        
        # Sauvegarde du fichier nettoyé
        df.to_csv(cleaned_output_path, index=False, encoding="utf-8")
        print(f" Fichier sauvegardé après nettoyage : {os.path.basename(cleaned_output_path)}")

    except Exception as e:
        print(f" Erreur lors du nettoyage de {file_name} : {e}")
        continue


 Nettoyage du fichier : benin_events_20250421_221827.csv
 Lignes supprimées (valeurs manquantes) : 3660 (reste 464982)
 Lignes supprimées (doublons) : 1 (reste 464981)
 Fichier sauvegardé après nettoyage : benin_events_20250421_221827_cleaned.csv

 Nettoyage du fichier : benin_mentions_20250421_221838.csv
 Lignes supprimées (valeurs manquantes) : 90 (reste 796614)
 Lignes supprimées (doublons) : 3495 (reste 793119)
 Fichier sauvegardé après nettoyage : benin_mentions_20250421_221838_cleaned.csv


In [54]:
# Liste des fichiers dans le dossier cleaned_csv_dir
dfs = {}    # Dictionnaire pour stocker les DataFrames _cleaned

for file_name in os.listdir(cleaned_csv_dir):
    
    # Vérification si c'est un fichier CSV
    if file_name.lower().endswith('.csv'):
        input_path = os.path.join(cleaned_csv_dir, file_name)
        
        try:
            try:
                # Chargement du DataFrame
                df = pd.read_csv(input_path)
                dfs[file_name] = df
                print(f" Fichier chargé {file_name} : ({len(df)} lignes)")
            except Exception as e:
                print(f"Erreur lecture {file_name} : {e}")
                
            # Afficher les types des colonnes avant modification
            print("\n Types des colonnes avant modification :")
            print(df.dtypes)

        except Exception as e:
            print(f" Erreur pour l'affichage du type des colonnes le fichier {file_name} : {e}")

 Fichier chargé benin_mentions_20250421_221838_cleaned.csv : (793119 lignes)

 Types des colonnes avant modification :
GLOBALEVENTID          int64
MentionTimeDate        int64
MentionType            int64
MentionSourceName     object
MentionIdentifier     object
ActionCharOffset       int64
Confidence             int64
MentionDocTone       float64
dtype: object
 Fichier chargé benin_events_20250421_221827_cleaned.csv : (464981 lignes)

 Types des colonnes avant modification :
GLOBALEVENTID           int64
SQLDATE                 int64
GoldsteinScale        float64
NumMentions             int64
NumSources              int64
NumArticles             int64
AvgTone               float64
ActionGeo_FullName     object
ActionGeo_Lat         float64
ActionGeo_Long        float64
SOURCEURL              object
dtype: object


In [22]:
# Définir le dossier de sauvegarde des fichiers traités
csv_output = "/home/pionner02/Pionner02 UlChris-Project/data"

In [ ]:
# Liste des fichiers dans le dossier cleaned_csv_dir
for file_name, df in dfs.items():
        
        print(f"\n Formatage pour le fichier {file_name} ")
        try:
            # Formatage pour le fichier contenant "benin_events"
            if "benin_events" in file_name:
                if "SQLDATE" in df.columns:
                    df["SQLDATE"] = pd.to_datetime(df["SQLDATE"], format='%Y%m%d', errors='coerce').dt.strftime("%Y-%m-%d")
                    print(f" Colonne 'SQLDATE' formatée au format YYYY-MM-DD.")

            # Formatage pour le fichier contenant "benin_mentions"
            if "benin_mentions" in file_name:
                if "MentionTimeDate" in df.columns:
                    df["MentionTimeDate"] = pd.to_datetime(df["MentionTimeDate"], format='%Y%m%d%H%M%S', errors='coerce').dt.strftime("%Y-%m-%d %H:%M:%S")
                    print(f" Colonne 'MentionTimeDate' formatée au format YYYY-MM-DD HH:MM:SS.")

            # Définir nom du fichier traité
            base_name, ext = os.path.splitext(file_name)  # séparer nom et extension
            csv_file_name = f"{base_name}_treated{ext}"

            # Destination des fichiers après traitement
            csv_output_path = os.path.join(csv_output, csv_file_name)
            
            # Sauvegarde des fichiers traités
            df.to_csv(csv_output_path, index=False, encoding="utf-8")
            print(f" Fichier sauvegardé après traitement dans : {os.path.basename(csv_output_path)}")

        except Exception as e:
            print(f" Erreur lors du traitement de {file_name} : {e}")



 Formatage pour le fichier benin_mentions_20250421_221838_cleaned.csv 
 Colonne 'MentionTimeDate' formatée au format YYYY-MM-DD HH:MM:SS.
 Fichier sauvegardé après traitement dans : benin_mentions_20250421_221838_cleaned_treated.csv

 Formatage pour le fichier benin_events_20250421_221827_cleaned.csv 
 Colonne 'SQLDATE' formatée au format YYYY-MM-DD.
 Fichier sauvegardé après traitement dans : benin_events_20250421_221827_cleaned_treated.csv


In [57]:
for file_name in os.listdir(csv_output):
    if file_name.lower().endswith('.csv'):
        file_path = os.path.join(csv_output, file_name)
        
        try:
            # Chargement du DataFrame
            df = pd.read_csv(file_path)
            if 'events' in file_name or 'mentions' in file_name:
                print(f"\n Types des colonnes après modification dans {file_name} : ")
                cols_to_check = ["SQLDATE", "MentionTimeDate"]
            
            # Vérification du type des colonnes après modification
            for col in cols_to_check:
                if col in df.columns:
                    print(f" - {col} : {df[col].dtype}")

        except Exception as e:
            print(f" Erreur de lecture du fichier {file_name} : {e}")



 Types des colonnes après modification dans benin_events_20250421_221827_cleaned_treated.csv : 
 - SQLDATE : object

 Types des colonnes après modification dans benin_mentions_20250421_221838_cleaned_treated.csv : 
 - MentionTimeDate : object


In [17]:
# Définir le dossier de destination de la base de données
output_db_dir = "/home/pionner02/Pionner02 UlChris-Project/data"

print(f" La base de données sera sauvegardée dans : {output_db_dir}")

 La base de données sera sauvegardée dans : /home/pionner02/Pionner02 UlChris-Project/data


In [23]:
import sqlite3

# Chemin vers le fichier SQLite qui contiendra la base de données
db_path = os.path.join(output_db_dir, 'gdelt_benin.db')

try:
    # Connexion à la base de données SQLite
    conn = sqlite3.connect(db_path)

    # Création d’un curseur pour exécuter des commandes SQL
    cursor = conn.cursor()

    # Optimisations de la base de données SQLite pour de meilleures performances
    
    cursor.execute("PRAGMA journal_mode = OFF;")  # Désactiver les journaux
    cursor.execute("PRAGMA synchronous = OFF;")  # Réduire la synchronisation (plus rapide mais moins sûr)
    cursor.execute("PRAGMA cache_size = 10000;")  # Augmenter la taille du cache
    cursor.execute("PRAGMA temp_store = MEMORY;")  # Utiliser la mémoire pour les tables temporaires
    cursor.execute("PRAGMA foreign_keys = OFF;") # Désactiver les clés étrangères pendant les insertions massives pour accélérer le processus
    
        
    # Supprimer les tables si elles existent et les recréer 

    cursor.executescript("""
        -- Supprimer les tables existantes si elles existent
        DROP TABLE IF EXISTS events;
        DROP TABLE IF EXISTS mentions;

        -- Créer la table events
        CREATE TABLE events (
            GLOBALEVENTID INTEGER PRIMARY KEY,         -- ID unique de l'événement
            SQLDATE INTEGER,                           -- Date au format YYYYMMDD
            GoldsteinScale REAL,                       -- Intensité de l’événement (échelle Goldstein)
            NumMentions INTEGER,                       -- Nombre total de mentions de l’événement dans les médias
            NumSources INTEGER,                        -- Nombre de sources distinctes mentionnant l’événement
            NumArticles INTEGER,                       -- Nombre d’articles différents mentionnant l’événement
            AvgTone REAL,                              -- Tonalité moyenne de l’événement
            ActionGeo_FullName TEXT,                   -- Nom complet de la localisation de l’action
            ActionGeo_Lat REAL,                        -- Latitude de la localisation de l’action
            ActionGeo_Long REAL,                       -- Longitude de la localisation de l’action
            SOURCEURL TEXT                             -- URL de la source originale
            );

        -- Créer la table mentions
        CREATE TABLE mentions (
            GLOBALEVENTID INTEGER,                      -- Référence à l’ID de l'événement
            MentionTimeDate INTEGER,                    -- Timestamp de la mention
            MentionSourceName TEXT,                     -- Nom de la source de la mention
            MentionIdentifier TEXT PRIMARY KEY,         -- Identifiant unique de la mention
            ActionCharOffset INTEGER,                   -- Position du mot déclencheur de l’action dans le texte
            Confidence INTEGER,                         -- Score de confiance de la mention)
            MentionDocTone REAL,                        -- Tonalité du document de mention
            FOREIGN KEY (GLOBALEVENTID) REFERENCES events(GLOBALEVENTID)  -- Clé étrangère liant la table mentions à la table events
            );
        """)
        
        # Message de confirmation pour indiquer que la base de données et les tables sont créées
    print("Base de données et tables créées avec succès.")
        
        # Valider création des tables
    conn.commit()

except sqlite3.OperationalError as e:
    # Message en cas d'échec du processus
    print(f" Erreur lor de la création de la base de données : {e}")

Base de données et tables créées avec succès.


In [24]:
# Nouveau dictionnaire pour les fichiers _treated les plus récents
treated_files_dict = {}

try:
    for prefix in ['benin_events', 
                   'benin_mentions'
                   ]:
        latest_file = get_latest_file_by_keyword(prefix, csv_output, ".csv")
        if latest_file:
            treated_files_dict[prefix] = os.path.join(csv_output, latest_file)
        
except Exception as e:
    print(f" Une erreur est survenue pendant la recherche des fichiers traités : {e}")

 Fichier le plus récent trouvé dans /home/pionner02/Pionner02 UlChris-Project/data pour 'benin_events' : benin_events_20250421_221827_cleaned_treated.csv
 Fichier le plus récent trouvé dans /home/pionner02/Pionner02 UlChris-Project/data pour 'benin_mentions' : benin_mentions_20250421_221838_cleaned_treated.csv


In [25]:
import pandas as pd

try:
    # Commencer une transaction pour les insertions massives
    conn.execute("BEGIN")

    # Charger les CSV dans les tables
    for table_name in [
                       'events', 
                       'mentions' 
                      ]:
        
        # Utiliser le fichier le plus récent pour chaque table
        csv_file = treated_files_dict.get(f'benin_{table_name}')
        
        # Si un fichier est manquant pour une table, passer à la table suivante
        if not csv_file:
            print(f"Fichier manquant pour {table_name}.")
            continue  # Passer à la table suivante si le fichier est manquant
        
        print(f" Chargement de {csv_file} dans la table '{table_name}'...")

        try:
            # Lire le CSV dans un DataFrame
            df = pd.read_csv(csv_file)
            
            # Vérification basique si le fichier est vide
            if df.empty:
                print(f" Le fichier {csv_file} est vide. Aucune donnée insérée.")
                continue
            
            # Insérer dans SQLite en écrasant les anciennes données
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            
            # Vérification du nombre de lignes après insertion
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            print(f"Nombre de lignes dans {table_name} après insertion : {count}")
        
        # Gestion des erreurs lors du chargement des fichiers
        except Exception as e:
            print(f" Erreur lors du chargement de {csv_file} dans {table_name}: {e}")
            continue # Passer à la table suivante en cas d'erreur

    # Finaliser la transaction
    conn.commit()
    print("Fin de la transaction avec succès.")

except Exception as e:
    conn.rollback()
    print(f" Erreur pendant la transaction, rollback effectué : {e}")

 Chargement de /home/pionner02/Pionner02 UlChris-Project/data/benin_events_20250421_221827_cleaned_treated.csv dans la table 'events'...


Nombre de lignes dans events après insertion : 464981
 Chargement de /home/pionner02/Pionner02 UlChris-Project/data/benin_mentions_20250421_221838_cleaned_treated.csv dans la table 'mentions'...
Nombre de lignes dans mentions après insertion : 793119
Fin de la transaction avec succès.


In [ ]:
cursor.execute("PRAGMA journal_mode = WAL;") # Réactiver les journaux
cursor.execute("PRAGMA synchronous = NORMAL;") # Réactiver la synchronisation
cursor.execute("PRAGMA foreign_keys = ON;") # Réactiver les clés étrangères

# Exécution des requêtes SQL pour créer des index supplémentaires
cursor.executescript("""
CREATE INDEX IF NOT EXISTS idx_events_date ON events(SQLDATE);                        -- Index sur la colonne SQLDATE de la table events pour accélérer les requêtes basées sur la SQLDATE

CREATE INDEX IF NOT EXISTS idx_mentions_event ON mentions(GLOBALEVENTID);             -- Index sur la colonne GLOBALEVENTID de la table mentions pour accélérer les jointures avec events

CREATE INDEX IF NOT EXISTS idx_mentions_mentiontimedate ON mentions(MentionTimeDate); -- Index sur la colonne MentionTimeDate de la table mentions pour accélérer les requêtes basées sur la MentionTimeDate

CREATE INDEX IF NOT EXISTS idx_events_actiongeo ON events(ActionGeo_FullName);  -- Index sur la colonne ActionGeo_FullName de la table events pour accélérer les recherches par localisation de l'action
                     
CREATE INDEX IF NOT EXISTS idx_mentions_source ON mentions(MentionSourceName);  -- Index sur la colonne MentionSourceName de la table mentions pour accélérer les recherches par nom de source

""")

# Affichage d'un message pour indiquer que les index ont été créés avec succès
print("Index créés avec succès.")

# Valider les modifications
conn.commit()
print("Base de données remplie et optimisée avec succès.")

# Fermeture de la connexion à la base de données SQLite après avoir terminé les opérations
conn.close()
print("\n Connexion à la base fermée.")


Index créés avec succès.
Base de données remplie et optimisée avec succès.


In [ ]:

from contextlib import contextmanager

# Vérification de l'existence de la base de données
if not os.path.exists(db_path):
    print(f" Fichier introuvable : {db_path}")
else:
    # Gestion de la connexion avec context manager
    try:
        with sqlite3.connect(db_path) as conn:
            print(f" Connexion réussie à : {db_path}")

            # Liste les tables de la base de données
            try:
                tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
                tables_df = pd.read_sql_query(tables_query, conn)
                print(" Tables disponibles :")
                display(tables_df)
            except Exception as e:
                print(f" Erreur en listant les tables : {e}")

            # Prévisualisation de la table 'events'
            try:
                print("\n Prévisualisation de la table 'events' (10 premières lignes) :")
                events_df = pd.read_sql_query("SELECT * FROM events LIMIT 10;", conn)
                display(events_df)
            except Exception as e:
                print(f" Erreur lors de la lecture de la table 'events' : {e}")

            # Prévisualisation de la table 'mentions'
            try:
                print("\n Prévisualisation de la table 'mentions' :")
                mentions_df = pd.read_sql_query("SELECT * FROM mentions LIMIT 10;", conn)
                display(mentions_df)
            except Exception as e:
                print(f" Erreur lors de la lecture de la table 'mentions' : {e}")

    except sqlite3.Error as e:
        print(f" Erreur de connexion : {e}")
    finally:
        print("\n Connexion à la base fermée.")

 Connexion réussie à : /home/pionner02/Pionner02 UlChris-Project/data/gdelt_benin.db
 Tables disponibles :


,name
0,events
1,mentions



 Prévisualisation de la table 'events' (10 premières lignes) :


,GLOBALEVENTID,SQLDATE,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
0,1238864555,2025-04-19,-10.0,15,2,15,-9.746639,"Koudou, Benin (general), Benin",10.8945,1.16328,https://guardian.ng/news/suspected-jihadists-k...
1,1239269408,2025-04-21,-10.0,16,2,16,-3.942652,Benin,9.5000,2.25000,https://www.aljazeera.net/news/2025/4/21/%D8%A...
2,1117208340,2023-07-26,-2.0,30,3,30,-4.848485,Benin,9.5000,2.25000,https://www.varmatin.com/faits-divers/qui-etai...
3,1117210338,2023-07-26,3.4,12,1,12,4.403409,Benin,9.5000,2.25000,https://www.prnewswire.com/news-releases/nexge...
4,1124580946,2023-09-03,3.4,12,2,12,2.535821,Benin,9.5000,2.25000,https://www.naijanews.com/2023/09/03/edo-obase...
5,1172759021,2024-04-29,0.0,14,2,14,3.384409,Nigeria,10.0000,8.00000,https://promptnewsonline.com/gov-obaseki-appro...
6,1061506981,2022-09-04,7.4,12,2,12,-0.966184,Benin,9.5000,2.25000,https://www.bignewsnetwork.com/news/272721277/...
7,1061507258,2022-09-04,0.0,12,2,12,-0.966184,Benin,9.5000,2.25000,https://www.bignewsnetwork.com/news/272721277/...
8,1068801089,2022-10-23,3.4,9,1,9,7.226107,Benin,9.5000,2.25000,http://nigerianobservernews.com/2022/10/confer...
9,1187563634,2024-07-17,2.8,16,1,16,-0.387597,Benin,9.5000,2.25000,http://malijet.com/actualite-sur-afrique/29291...



 Prévisualisation de la table 'mentions' :


,GLOBALEVENTID,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,ActionCharOffset,Confidence,MentionDocTone
0,1080064261,2023-01-13 20:00:00,1,bbc.com,https://www.bbc.com/news/av/world-africa-64269446,71,90,6.382979
1,1176169626,2024-05-16 18:45:00,1,tribuneonlineng.com,https://tribuneonlineng.com/edo-2024-inec-to-b...,417,90,1.176471
2,1001321613,2021-08-27 21:15:00,1,ahram.org.eg,https://english.ahram.org.eg/NewsContent/1/64/...,367,90,1.923077
3,1080210460,2023-01-15 14:30:00,1,english.news.cn,https://english.news.cn/20230115/8600befffbc44...,391,90,7.572816
4,1080210533,2023-01-15 14:30:00,1,english.news.cn,https://english.news.cn/20230115/8600befffbc44...,391,90,7.572816
5,1080214582,2023-01-15 01:15:00,1,china.org.cn,http://french.china.org.cn/foreign/txt/2023-01...,1533,90,1.333333
6,1080214730,2023-01-15 01:15:00,1,china.org.cn,http://french.china.org.cn/foreign/txt/2023-01...,1533,90,1.333333
7,1080214582,2023-01-15 01:15:00,1,china.org.cn,http://french.china.org.cn/foreign/txt/2023-01...,178,90,-0.895522
8,1080214730,2023-01-15 01:15:00,1,china.org.cn,http://french.china.org.cn/foreign/txt/2023-01...,178,90,-0.895522
9,1176686597,2024-05-19 20:30:00,1,nigerianobservernews.com,https://nigerianobservernews.com/2024/05/obase...,1013,90,2.749141



 Connexion à la base fermée.
